In [ ]:
import qiskit.quantum_info as qi
import math
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, Aer, transpile, assemble, execute,QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, array_to_latex
import networkx as nx 
from numpy import pi
from time import time
from qiskit.transpiler.passes import RemoveBarriers
import scipy as sc
import scipy.linalg as la
import json



import warnings
warnings.filterwarnings('ignore')

class myqc(QuantumCircuit):
    def set_ct(self,cq,tq):
        self.cq=cq
        self.tq=tq
        
    def paulis(self,qi,x):  
        if x==[0,0]:
            self.id(qi)
                     
        elif x==[0,1]:
            self.z(qi)
                     
        elif x==[1,0]:
            self.x(qi)
        elif x==[1,1]:
            self.y(qi)

# randomly generate coefficients of Ising Hamiltonian
def generate_ising_weight(N):
    edge_weight = []
    node_weight = []
    for i in range(N):
        weight = 2*np.random.random()-1
        node_weight.append(weight)
    
    for i in range(N):
        weight = 2 * np.random.random() -1 
        edge_weight.append(weight)

    return node_weight, edge_weight

# represent a Ising model as a graph
def ising_model(N,node_weight,edge_weight):
    # ising chain
    G = nx.Graph()

    for i in range(N):
        G.add_node(i,weight = node_weight[i])
    
    for i in range(N):
        G.add_edge(i,(i+1)%N,weight =edge_weight[i])
        
    return G

# the evolution unitary of a Ising Hamiltonian; right now valid for even 'N' only
def ising_U(G, gamma=1):
    N= len(G.nodes())
    circ = QuantumCircuit(N)

    for i in range(N):
        circ.rz(2*G.nodes[i]['weight'] * gamma, i)
    
    circ.barrier()

    for i in np.arange(0,N-1,2):
        circ.cnot(i,i+1)
        circ.rz(2 * G.get_edge_data(i,i+1)['weight'] * gamma,i+1)
        circ.cnot(i,i+1)
    circ.barrier()

    for i in np.arange(1,N-1,2):
        circ.cnot(i,i+1)
        circ.rz(2 * G.get_edge_data(i,i+1)['weight'] * gamma,i+1)
        circ.cnot(i,i+1)

    if N % 2 != 0:
        circ.barrier()

    circ.cnot(N-1,0)
    circ.rz(2 * G.get_edge_data(0,N-1)['weight'] * gamma,0)
    circ.cnot(N-1,0)

    circ.barrier()
    
    return circ 

# the circuit (in some standard form) representing the evolution unitary operator of Ising model; right now for even 'N' only
def ising_circ(G, gamma=1):
    N= len(G.nodes())
    circ = QuantumCircuit(N)

    for i in range(N):
        circ.u3(0,0,2*G.nodes[i]['weight'] * gamma, i)
    for i in np.arange(0,N-1,2):
        circ.cnot(i,i+1)
    circ.barrier()
    
    for i in np.arange(0,N,2):
        circ.u3(0,0,0,i)
        circ.u3(0,0,2 * G.get_edge_data(i,i+1)['weight'] * gamma,i+1)
        circ.cnot(i,i+1)
    for i in np.arange(1,N,2):
        circ.cnot(i,(i+1)%N)
    circ.barrier()
    
    for i in np.arange(1,N,2):
        circ.u3(0,0,0,i)
        circ.u3(0,0,2 * G.get_edge_data(i,(i+1)%N)['weight'] * gamma,(i+1)%N)
    for i in np.arange(1,N,2):   
        circ.cnot(i,(i+1)%N)
    circ.barrier()
    
    return circ 

# compute the energy of a Ising model under an eigenstate
def ising_energy(G, bit_str):

    state = [(-1)**int(k) for k in bit_str]
    E = 0 
    for (i,j) in G.edges:
        E = E+ state[i] * state[j] * G.get_edge_data(i,j)['weight']  
    for i in G.nodes:
        E = E+ state[i] * G.nodes[i]['weight']

    return E

# compute the eigen-phase of evolution unitary operator under an eigenstate
def ising_phase(G, bit_str, gamma=1):
    E= ising_energy(G, bit_str)
    phase= -1*E*gamma

    return phase%(2*np.pi)

    
# a randomly generated Ising Hamiltonian
nq = 10
gamma = 1
#node_weight , edge_weight  = generate_ising_weight(nq)
node_weight=[-0.31412885882803354, 0.4644558298187702, -0.5839604427195664, 0.31324845968060666, 0.8970580614419434, -0.37272591599690985, 0.24336920691141173, 0.37088138954096084, -0.37891887987773476, -0.8424226108147614]
edge_weight=[0.7022899024933575, 0.5067888956532058, 0.09884717980161595, 0.5165292853672563, 0.6268524049174844, -0.9667902292092547, -0.5273001456376085, 0.1108023701778269, -0.047125461867228546, -0.19656900108940256]
print('node_weight=',node_weight)
print('edge_weight=',edge_weight)
ising_h = ising_model(nq,node_weight, edge_weight)
ising_circuit = ising_circ(ising_h)


# define noise model for simulation
from qiskit_aer.noise import NoiseModel, amplitude_damping_error, depolarizing_error, coherent_unitary_error,phase_damping_error
from scipy.optimize import curve_fit, minimize
from qiskit.quantum_info import process_fidelity, PTM, diamond_norm

def unitary_error_rx(theta):  #exp(-i theta/2 X)
    return np.array([[np.cos(theta/2), -1j*np.sin(theta/2)], [-1j*np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_rxx(theta):          #exp(-i theta/2 X\otimes X)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,1],[0,0,1,0],[0,1,0,0],[1,0,0,0]])

def unitary_error_rz(theta):  #exp(-i theta/2 Z)
    return np.array([[np.exp(-1j*theta/2), 0], [0, np.exp(1j*theta/2)]])

def unitary_error_rzz(theta):          #exp(-i theta/2 Z\otimes Z)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[1,0,0,0],[0,-1,0,0],[0,0,-1,0],[0,0,0,1]])

def unitary_error_ry(theta):  #exp(-i theta/2 Y)
    return np.array([[np.cos(theta/2), -np.sin(theta/2)], [np.sin(theta/2), np.cos(theta/2)]])

def unitary_error_ryy(theta):          #exp(-i theta/2 Y\otimes Y)
    return np.cos(theta/2)*np.eye(4)-1j*np.sin(theta/2)*np.array([[0,0,0,-1],[0,0,1,0],[0,1,0,0],[-1,0,0,0]])

def fsim_error(theta,phi): # unitary error of fsim kind
    mat= [[1, 0, 0, 0],[0, np.cos(theta), -1j*np.sin(theta),0], [0,-1j*np.sin(theta),np.cos(theta),0],[0,0,0,np.exp(1j*phi)]]
    return mat

# generate unitary noise
def get_noise_model(ps=0.001,angle=[0,0]):
    noise_model=NoiseModel()
    
    error_ad= amplitude_damping_error(ps)
    error_pd= phase_damping_error(ps)
    error_st= error_pd.compose(error_ad)
    
    error_st_2q= error_st.tensor(error_st)
    
    error_u_mat= fsim_error(*angle)
    error_u = coherent_unitary_error(error_u_mat)
    
    error_1q= error_st
    error_2q= error_u.compose(error_st_2q)
    noise_model.add_all_qubit_quantum_error(error_1q, ['s','sdg','t','h','u1','u2','u3','I','X','Y','Z','ry','rz','rx'])
    noise_model.add_all_qubit_quantum_error(error_2q, ['cz','cx','fsim'])
    
    return noise_model,error_1q,error_2q

# simulate circuits with given noise model
def noise_sim(circ_list,noise_model,shots=10**4):
    backend = Aer.get_backend("qasm_simulator")
    backend.set_options(device='GPU')
    nq= circ_list[0].num_qubits
    result = execute(circ_list, backend= backend,noise_model=noise_model, shots=shots,optimization_level=0).result()
    count_list=[result.get_counts(circ) for circ in circ_list]
    exp_list=[count.get('0'*nq,0)/shots for count in count_list]
    
    return exp_list


# repeat a circuit for k times
def circ_rep(k,circ):
    num_qubits= circ.num_qubits
    U_rep=myqc(num_qubits)
    for i in range(k):
        U_rep=U_rep.compose(circ)

    return U_rep

# transform a circuit to some standard form which is convenient for randomized compiling
def standard_circ(circ):
    circ= RemoveBarriers()(circ)
    circ= transpile(circ,basis_gates=['u3','cx'])
    qc= QuantumCircuit(circ.data[0][1][0].register)
    index=1
    for ins in circ:
        ins_nq= ins[0].num_qubits
        if ins_nq< index:
            qc.barrier()
        qc.append(*ins)
        index= ins_nq
    return qc

#functions for RC
def divide_circ(circ):
    circ_list=[[]]
    ind=0
    for ins in circ[0:-1]:
        if 'barrier' == ins[0].name:
            ind=ind+1
            circ_list.append([])
        else:
            circ_list[ind].append(ins)
    circ_list_p=[]
    for cycle in circ_list:
        if len(cycle)!=0:
            circ_list_p.append(cycle)
    return circ_list_p        

def prod_pauli(plist):
    pauli_list=np.array([[0,0],[0,1],[1,0],[1,1]])
    s=np.array([0,0])
    for q in plist:
        s=s+pauli_list[q]
    return [i%2 for i in s]

def cx_on_pauli(p1,p2):
    
    cx=[[1,0,0,0],[1,1,0,0],[0,0,1,1],[0,0,0,1]]
   
    pauli_2q=[p1[0],p2[0],p1[1],p2[1]]
    new_pauli=np.mod(np.dot(cx,pauli_2q),2)
    new_pauli1=[int(new_pauli[0]),int(new_pauli[2])]
    new_pauli2=[int(new_pauli[1]),int(new_pauli[3])]
    return new_pauli1,new_pauli2

def u3_pauli(u3,p):    #the product of u3 * p
    new_u3= (u3[0].copy(),u3[1],u3[2])
    [theta, phi, lam]= [float(x) for x in new_u3[0].params]
    if p==[0,1]:      #p=z
        lam= (lam + np.pi)%(2*np.pi)
    elif p==[1,0]:      #p=x
        theta= (theta+ np.pi)%(2*np.pi)
        lam= (np.pi-lam)%(2*np.pi)
    elif p==[1,1]:                    #p=y
        theta= (theta+ np.pi)%(2*np.pi)
        lam= -lam
    new_u3[0].params=[theta, phi, lam]
    return new_u3

def pauli_u3(u3,p):    #the product of  p*u3
    new_u3= (u3[0].copy(),u3[1],u3[2])
    [theta, phi, lam]= [float(x) for x in new_u3[0].params]
    if p==[0,1]:      #p=z
        phi= (phi + np.pi)%(2*np.pi)
    elif p==[1,0]:      #p=x
        theta= (theta+ np.pi)%(2*np.pi)
        phi= (np.pi-phi)%(2*np.pi)
    elif p==[1,1]:                    #p=y
        theta= (theta+ np.pi)%(2*np.pi)
        phi= -phi
    new_u3[0].params=[theta, phi, lam]
    return new_u3


def circ_q1_compile(cor_paulis,g1_list,rand_paulis,nq=10):
    g1_list_new=[]
    for qb in range(nq):
        new_u3=u3_pauli(g1_list[qb],cor_paulis[qb])
        new_u3= pauli_u3(new_u3,rand_paulis[qb])
        g1_list_new.append(new_u3)   
    
    return g1_list_new


def compiled_cycle(bare_cycle,cor_paulis0,nq=10):
    qc = myqc(nq)
    for i in range(nq):
        qc.u3(0,0,0,i)
    g1_list=[ins for ins in qc]
    g2_list=[]
    for ins in bare_cycle:
        if ins[0].num_qubits==1:
            ind=ins[1][0].index
            g1_list[ind]=ins
        else:
            g2_list.append(ins)
    
    pauli_list=[[0,0],[0,1],[1,0],[1,1]] 
    pindex= np.random.randint(4,size=nq)
    rand_paulis= [pauli_list[pindex[i]] for i in range(nq)]
    compiled_ops= circ_q1_compile(cor_paulis0,g1_list,rand_paulis,nq=nq)

    cor_paulis1=rand_paulis.copy()
    
    for ins in g2_list:
        ind1, ind2= (qubit.index for qubit in ins[1])
        cor_paulis1[ind1],cor_paulis1[ind2]= cx_on_pauli(cor_paulis1[ind1],cor_paulis1[ind2])
        
    return compiled_ops,g2_list,cor_paulis1

# the core function for RC
def randomized_compiling(bare_circ,nq=10):
    circ1=divide_circ(bare_circ)
    
    qc=myqc(nq)
    cor_paulis0= [[0,0] for i in range(nq)]
    for cycle in circ1:
        compiled_ops,g2_list,cor_paulis1=compiled_cycle(cycle,cor_paulis0,nq)
        
        for i in range(nq):
            qc.append(*compiled_ops[i])
        if g2_list!=[]:
            for ins in g2_list:
                qc.append(*ins)
        
        qc.barrier()
        cor_paulis0= cor_paulis1
    
    for i in range(nq):
        qc.paulis(i,cor_paulis0[i])
    
    return qc


# compute the actual process infidelity of a noise channel
def act_error(error):
    pf= process_fidelity(error)
    return 1-pf

#compute the actual process infidelity of a circuit under a noise model; note this function can only be used for a cirucit with several qubits
def actual_circ_error(noise_model,circ):
    circ_tar=circ.copy()
    circ_tar.save_superop()
    simulator= Aer.get_backend('aer_simulator_superop')
    ideal_sop= simulator.run(circ_tar).result()
    noisy_sop= simulator.run(circ_tar,noise_model=noise_model).result()
    error= 1-process_fidelity(noisy_sop.data()['superop'],ideal_sop.data()['superop'])
    return error


#define matrix pencil data processing technique; this code is from the paper 'spectral quantum tomography' with some modifications
def matrix_pencil(time_series_data,L ,N_poles,cutoff=10**(-2)):
        """Computes a decomposition into exponentially decaying oscillations of a given time series.
        Input: time_series_data: a list of floats corresponding to the state of the system at fixed time intervals.
                NOTE: It is important this timeseries starts at t=0 (k=0), the method as implemented can't deal with timeshifts and may fail quietly
               L: A matrix pencil parameter. Set between 1/2 and 2/3 of len(time_series_data) 
               N_poles: The number of maximum possible poles the data can be decomposed into. Choosing this number too small will lead to bad fits so act with care.
               cutoff: a cut-off for the smallest possible relative amplitude a poles can contribute with, standard is 10^(-2).
        Ouput: poles: a scipy array of the poles (the oscillating bits).
                amplitudes: a scipy array of amplitudes corresponding to the poles.
                
                """
        
        #Compute the length of the data series and store it in N
        N = len(time_series_data)

        #Compute the Hankel matrix of the data
        Y = sc.matrix([time_series_data[i:L+i+1] for i in range(0,N-L)]) 
        
        #Take the singular value decomposition of the data Hankel matrix
        U,S,Vh = sc.linalg.svd(Y) 
        Vh =sc.matrix(Vh)
        U = sc.matrix(U)
        N_tem= N_poles
        for i in range(2,N_tem):   # define the 'N_poels'
            if (S[i]/S[i+1])>5 and (S[i]/S[i+1])>(S[i+1]/S[i+2]):
                break
        N_poles =i+1
        #The ESPRIT method includes a filter step that gets rid of small singular values.
        # Since for us the number of poles is known I just retain the N_poles largest singular values and corresponding right eigenspace.
        #If N_poles is so large that it starts to include nonsense values we let the cutoff parameter set the number of relevant poles instead.
        #We choose to retain only the singular values s such that s>s_max * cutoff where s_max is the largest singualr value
        Scutoff = S[S>cutoff*S[0]]
        if len(Scutoff)<N_poles:

            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(len(Scutoff))] for j in range(N-L)])
            Vhprime = Vh[0:len(Scutoff),:]
        else:
            Sprime = sc.matrix([[S[i] if i==j else 0 for i in range(N_poles)] for j in range(N-L)])
            Vhprime = Vh[0:N_poles,:]
        
        #Compute the shifted matrices for the matrix pencil.
        Vhprime1 = Vhprime[:,0:-1]
        Vhprime2 = Vhprime[:,1:]
        
        #Compute the solution of the matrix pencil (via SVD)
        Y = la.pinv(Vhprime1.H)*Vhprime2.H
        poles, vecs = sc.linalg.eig(Y)
        for i in range(len(poles)):
            amp= np.abs(poles[i])
            if amp>1:
                poles[i]= poles[i]/amp
        
        #Compute the amplitudes by least squares optimization
        Z = sc.matrix([poles**k for k in range(N)])
        amplitudes = la.lstsq(Z,sc.matrix(time_series_data).transpose())
        ampls = sc.array([a[0] for a in amplitudes[0]])
        amp_max= np.max(np.abs(ampls))
        
        polesp=[]
        amplsp=[]
        for i in range(len(poles)):
            if np.abs(ampls[i])/amp_max>0.1:
                polesp.append(poles[i])
                amplsp.append(ampls[i])
                
        #return the poles and amplitudes as scipy arrays
        return polesp, amplsp,amplitudes,S

# process the results from matrix pencil method
def mp_est(poles,target_phase=1/3*pi,nrep=1):
    amps= np.abs(poles)
    phases= np.angle(poles)
    res=[]
    
    if np.abs(target_phase) > 10**-4:
        phase_difs= np.abs(np.abs(phases)-np.abs(target_phase))
        phase_error= np.min(phase_difs)
        angle_index= np.argmin(phase_difs)
        index_others= np.argwhere(phase_difs>phase_error)[:,0]

        res.append(phase_error/nrep)
        res.append((amps[angle_index])**(1/nrep))
        for index in index_others:
            res.append((poles[index])**(1/nrep))
        res.append(target_phase)
        
    else:
        phase_difs= np.abs(phases)
        angle_error= np.max(phase_difs)
        res.append(angle_error/nrep)
        for pole in poles:
            res.append(pole**(1/nrep))
        res.append(0)    
    return res

# get the process infidelity, stochastic infidelity, maximum phase error from the inputs of diagonal entries of noise channel
def result_mp(mp_est_list,nq=10,ndeg=0):
    dim= 2**nq
    dim_ts= dim + ndeg**2-ndeg
    dim_ns= dim**2-dim_ts
    ns_list=[]
    ts_list= []
    angle_list=[]
    
    for mp_est in mp_est_list:
        if np.abs(mp_est[-1])>10**-4:
            ns_list.append(mp_est[1]*np.exp(1j*mp_est[0]))
            ns_list.append(mp_est[1]*np.exp(-1j*mp_est[0]))
            if np.abs(mp_est[-1]-pi)<10**-4:
                for x in mp_est[2:-1]:
                    if np.abs(np.abs(np.angle(x))-pi)<0.5:
                        ns_list.append(-1*x)
                    else:
                        if np.abs(np.angle(x))<0.5:
                            ts_list.append(x)
            else:
                for x in mp_est[2:-1]:
                    if np.abs(np.angle(x))<0.5:
                        ts_list.append(x)          
        else:
            for x in mp_est[1:-1]:
                if np.abs(np.angle(x))<0.5:
                    ts_list.append(x)
                
    if len(ns_list)>=1:
        f_ns_mean= np.abs(np.mean(ns_list))
    else:
        f_ns_mean=1
    if len(ts_list)>=1:
        f_ts_mean= np.abs(np.mean(ts_list))
    else:
        f_ts_mean=1
    f_mean= (dim_ns*f_ns_mean+ dim_ts*f_ts_mean)/dim**2
    
    if len(ns_list)>=1:
        u_ns_mean= np.mean(np.square(np.abs(ns_list)))
    else:
        u_ns_mean=1
    if len(ts_list)>=1:
        u_ts_mean= np.mean(np.square(np.abs(ts_list)))
    else:
        u_ts_mean=1
    u_mean= (dim_ns*u_ns_mean+ dim_ts*u_ts_mean)/dim**2

    angle_list= np.concatenate((np.angle(ns_list),np.angle(ts_list)))
    angle_error= np.max(np.abs(angle_list))
    return [1-f_mean, 1-np.sqrt(u_mean),angle_error]


# transform an integer to a bit-string
def int2bin(x,nq=3):
    xb= bin(x)[2::]
    for i in range(nq-len(xb)):
        xb='0'+xb
    return xb

# generate a pair of random bit-strings
def random_states(nq=10):
    rs=np.random.choice(2**nq,2,replace=False)
    return [int2bin(rs[0],nq),int2bin(rs[1],nq)]

# the circuit for preparing initial state
def ini_circ(ini_states,bs_list=['z' for i in range(nq)]):
    s0, s1= ini_states
    nq= len(s0)
    ini_circ= myqc(nq,nq)
    ent_qbits=[]
    for n in range(nq):
        if s0[n]==s1[n]:
            if s0[n]=='1':
                ini_circ.u3(pi,-pi/2,pi/2,n) #x gate
            if bs_list[n]=='x':
                ini_circ.u3(pi/2,0,pi,n)    # h gate
        else:
            ent_qbits.append(n)

    neq= len(ent_qbits)
    if [s0[x] for x in ent_qbits].count('0')< [s1[x] for x in ent_qbits].count('0'):
        s0,s1= s1, s0
    
    if neq==1:
        if bs_list[ent_qbits[0]]=='z':
            ini_circ.u3(pi/2,0,pi,ent_qbits[0])  #h gate
    if neq>=2:
        ini_circ.u3(pi/2,0,pi,ent_qbits[0])  #h gate
        for n in range(neq-1):
            ini_circ.cx(ent_qbits[n],ent_qbits[n+1])
        
        ini_circ.barrier()       
        for n in ent_qbits:
            if s0[n]=='1':
                ini_circ.u3(pi,-pi/2,pi/2,n)  # x gate
            if bs_list[n]=='x':
                ini_circ.u3(pi/2,0,pi,n) # h gate
    return ini_circ    

# the circuit for measurement
def meas_circ(ini_states,bs_list=['z' for i in range(nq)]):
    s0, s1= ini_states
    nq= len(s0)
    ini_circ= myqc(nq,nq)
    ent_qbits=[]
    for n in range(nq):
        if s0[n]==s1[n]:
            if s0[n]=='1':
                ini_circ.u3(pi,-pi/2,pi/2,n) #x gate
            if bs_list[n]=='x':
                ini_circ.u3(pi/2,0,pi,n)    # h gate
        else:
            ent_qbits.append(n)

    neq= len(ent_qbits)
    if [s0[x] for x in ent_qbits].count('0')< [s1[x] for x in ent_qbits].count('0'):
        s0,s1= s1, s0
    
    if neq==1:
        if bs_list[ent_qbits[0]]=='z':
            ini_circ.u3(pi/2,0,pi,ent_qbits[0])  #h gate
        
    if neq>=2:      
        for n in ent_qbits:
            if s0[n]=='1':
                ini_circ.u3(pi,-pi/2,pi/2,n)  # x gate
            if bs_list[n]=='x':
                ini_circ.u3(pi/2,0,pi,n) # h gate
        for n in reversed(range(neq-1)):
            ini_circ.cx(ent_qbits[n],ent_qbits[n+1])
        
        ini_circ.barrier() 
        ini_circ.u3(pi/2,0,pi,ent_qbits[0])  #h gate
    return ini_circ   

# the circuit for csb
# 'len_list': the list of lengths of circuits; 'states': initial state; 'nrc': num of rand circs for RC
# 'rc': rc=0 not performing RC; rc=1 performing RC on SPAM only; rc=2 performing RC on full circuit
def benchmark_circ(len_list,states,nrc=1,rc=0,nq=10,target_circ=ising_circuit):
    circ_list=[]

    qc_ini= ini_circ(states)
    qc_ini.barrier()

    qc_meas= meas_circ(states)                          #qc_ini.inverse()
    qc_meas.barrier()

    for lc in len_list:
        for k in range(nrc):
            qc_rep= circ_rep(lc,target_circ)
            if rc==1:                                              # RC on SPAM only
                qc_ini_com= randomized_compiling(qc_ini,nq=nq)
                qc_ini_com.barrier()
                qc_meas_com= randomized_compiling(qc_meas,nq=nq)
                qc=myqc(nq,nq)
                qc= qc.combine(qc_ini_com)
                qc=qc.combine(qc_rep)
                qc= qc.combine(qc_meas_com)
                qc.barrier()
            elif rc==2:                                     # RC on whole circuit
                qc=myqc(nq,nq)
                qc= qc.combine(qc_ini)
                qc=qc.combine(qc_rep)
                qc= qc.combine(qc_meas)
                qc_com= randomized_compiling(qc,nq=nq)
                qc=myqc(nq,nq)
                qc=qc.combine(qc_com)
                qc.barrier()
            else:
                qc=qc_ini.combine(qc_rep)
                qc= qc.combine(qc_meas)
            qc.measure(range(nq),range(nq))
            circ_list.append(qc)
    return circ_list

# the full lists of circuits for csb
# 'states_list': list of initial states; 'len_max': the maximum length of circuits; 'nq': num of qubits
#  'nrc': num of rand circuits for RC; 'rc':whether performing RC; 
# nrep_list: the list of nums of repeatitions of the target circuit, the length of list should be the same as num of initial states
def csb_full_circs(states_list,len_max=50,nq=10,nrc=1,rc=0,nrep_list=[1 for i in range(10)]):
    circ_list=[]
    for k,states in enumerate(states_list):
        len_list=[i*nrep_list[k] for i in range(0,len_max+1)]
        circ_list.append(benchmark_circ(len_list,states,nrc=nrc,rc=rc,nq=nq))
    return circ_list

# results of csb
# 'data': measured probabilities of benchmarking circuits
def result(data,states_list,mp_cutoff=10**-2,N_poles=4,nrep_list=[1 for i in range(10)]):
    target_phases=[]
    for states in states_list:
        s0= ising_phase(ising_h,states[0])
        s1= ising_phase(ising_h,states[1])
        phase= (s0-s1)%(2*pi)
        if phase>pi:
            phase= phase-2*pi
        target_phases.append(phase)
    
    num_modes= len(states_list)
    mp_est_list=[]
    for i in range(num_modes):
        max_len= len(data[i])
        poles, amps,amplitudes,S = matrix_pencil(data[i],L= math.ceil(max_len*3/5),N_poles=N_poles,
        cutoff=mp_cutoff)
        mp_est_list.append(mp_est(poles,target_phase=target_phases[i],nrep= nrep_list[i]))
    results = result_mp(mp_est_list,nq=nq)
    return results,mp_est_list

# doing the simulation of csb
def simulation(noise_model,ns=10,len_max=50,shots=10**4,nq=10,nrc=1,rc=0,N_poles=4,mp_cutoff=10**-2,nrep_list=[1 for k in range(10)]):
    states_list= [random_states(nq=nq) for k in range(ns)]
    circs_list=csb_full_circs(states_list,len_max=len_max,nq=nq,nrc=nrc,rc=rc,nrep_list=nrep_list)
    data=[]
    for circs in circs_list:
        plist=noise_sim(circs,noise_model=noise_model,shots=shots)
        plist=list(map(np.mean,np.split(np.array(plist),len_max+1)))
        data.append(plist)
    res, spectrum=result(data,states_list,mp_cutoff=mp_cutoff,N_poles=N_poles,nrep_list=nrep_list)
    return res,spectrum,data,states_list

In [ ]:
# simulation with fixed unitary error and varied stochastic error
ns=10
nrc=1
rc=0
shots=10**4/nrc
ps_list=[0.001,0.002,0.005,0.007,0.01]
act_error_list=[]
nrep_list_list= [[1 for i in range(ns)] for k in range(5)]
len_max_list=[50,50,50,50,50]
mp_cutoff= 10**-2
N_poles=4
est_sto=[]
for i, ps in enumerate(ps_list):
    noise_model,error_1q,error_2q= get_noise_model(ps=ps,angle=[0.01,0.01])
    acter1=act_error(error_1q)
    acter2=act_error(error_2q)
    act_error_list.append(1-(1-acter1)**(3*nq)*(1-acter2)**(2*nq))
    for k in range(10):
        res, sp, data, states_list=simulation(noise_model=noise_model,ns=ns,nq=nq,len_max= len_max_list[i],shots=shots,
        nrc=nrc,rc=rc,mp_cutoff=mp_cutoff,N_poles=N_poles,nrep_list=nrep_list_list[i])
        est_sto.append(res)
print(est_sto)

In [ ]:
# simulation with fixed stochastic error and varied unitary error
ns=10
nrc=1
rc=0
shots=10**4/nrc
mp_cutoff= 10**-2
N_poles=4
angle_list=[0.01,0.03,0.05,0.07,0.1]
act_uni_error_list=[]
nrep_list_list= [[1 for i in range(ns)] for k in range(5)]
len_max_list=[50,50,50,50,50]
est_uni=[]
for i, angle in enumerate(angle_list):
    noise_model,error_1q,error_2q= get_noise_model(ps=0.001,angle=[angle,angle])
    acter1=act_error(error_1q)
    acter2=act_error(error_2q)
    act_uni_error_list.append(1-(1-acter1)**(3*nq)*(1-acter2)**(2*nq))
    for k in range(10):
        res, sp, data, states_list=simulation(noise_model=noise_model,ns=ns,nq=nq,len_max= len_max_list[i],shots=shots,
        nrc=nrc,rc=rc,N_poles=N_poles,mp_cutoff=mp_cutoff,nrep_list=nrep_list_list[i])
        est_uni.append(res)
print(est_uni)

In [ ]:
# simulation with randomized compiling (this code is very time consuming!)
ts=time()
ns=10
nrc=10
rc=2
shots=10**4/nrc
mp_cutoff= 10**-2
N_poles=4
angle_list=[0.01,0.03,0.05,0.07,0.1]
act_uni_error_list=[]
nrep_list_list= [[1 for i in range(ns)] for k in range(5)]
len_max_list=[50,50,50,50,50]
est_uni_rc=[]
for i, angle in enumerate(angle_list):
    noise_model,error_1q,error_2q= get_noise_model(ps=0.001,angle=[angle,angle])
    acter1=act_error(error_1q)
    acter2=act_error(error_2q)
    act_uni_error_list.append(1-(1-acter1)**(3*nq)*(1-acter2)**(2*nq))
    for k in range(10):
        res, sp, data,states_list=simulation(noise_model=noise_model,ns=ns,nq=nq,len_max= len_max_list[i],shots=shots,
        nrc=nrc,rc=rc,mp_cutoff=mp_cutoff,N_poles=N_poles,nrep_list=nrep_list_list[i])
        est_uni_rc.append(res)
print('actual error=',act_uni_error_list)
print('estimate=',est_uni_rc)
te= time()
print('time cost=', te-ts)